# Resumen Ejecutivo
Durante el proceso de revisión de los informes de práctica del DISC (Departamento de Ingeniería de Sistemas y Computación), se requiere una inversión considerable de tiempo que, hasta la fecha, no ha sido automatizada. Esto conlleva largas jornadas de trabajo y carga adicional para los académicos, quienes podrían emplear ese tiempo en otras labores. Por lo tanto, como equipo de trabajo, hemos llegado a un consenso en la necesidad de llevar a cabo el análisis y desarrollo de un modelo que permita clasificar los informes en las categorías definidas en la rúbrica actual (insatisfactorio, regular, bueno y excelente).
Es importante destacar que, con la llegada de la pandemia, la entrega de informes ha sido en formato digital, lo que ha generado un conjunto de aproximadamente 100 informes disponibles. Esta digitalización ofrece ventajas significativas para el entrenamiento del modelo, ya que se dispone de datos de entrada y resultados concretos (informe, rúbrica y nota).


In [25]:
import fitz
import pandas as pd

# Lectura de datos


In [2]:
def get_classification(grade, number=False):
  classification = [0,0,0] # Regular, Bueno, Excelente (Todo 0 = Insatisfactorio)
  grade = round(grade, 1)
  if(grade < 4):
    return "insatisfactorio" if not number else 0
  elif (4 <= grade < 5.5):
    classification[0] = 1
    return "regular" if not number else 1
  elif (5.5 <= grade < 6.5):
    classification[1] = 1
    return "bueno" if not number else 2
  elif (6.5 <= grade <= 7):
    classification[2] = 1
    return "excelente" if not number else 3

In [3]:
dataset = pd.read_excel("calificaciones.xlsx", decimal=',')
grades_columns = dataset.columns.difference(["id", "periodo", "Unnamed: 9"]) #["estructura", "escritura", "contenido", "conclusiones", "conocimiento", "relevancia", "total"]
rubric_columns = grades_columns.difference(["total"]) #, "escritura", "estructura"
dataset = dataset.dropna(subset=grades_columns)

# Extracción y limpieza de documentos
En esta sección, se cargan los documentos en formato PDF, para la extracción y limpieza de estos, seguido de su integración al dataset.

In [4]:
documents = []

for id in dataset['id']:
    pdf_file = fitz.open(f"dataset/{id}.pdf")
    document_text = chr(12).join([page.get_text() for page in pdf_file])
    documents.append(document_text)
dataset.insert(loc=2, column="documents", value=documents)
dataset

,id,periodo,documents,estructura,escritura,contenido,conclusiones,conocimiento,relevancia,total,Unnamed: 9
0,20908397-1,2023-1,\n \nUNIVERSIDAD CATÓLICA DEL NORTE \nFACUL...,6.2,5.1,6.0,5.5,4.4,6.0,5.3,NaN
1,18971994-1,2023-1,\nAntofagasta \n \n Abril de 2023 \...,6.9,6.8,6.8,6.8,7.0,6.9,6.9,NaN
2,19445943-1,2023-1,\n1 \n \n \nUNIVERSIDAD CATÓLICA DEL NORTE \n...,6.7,6.9,6.5,6.4,6.8,7.0,6.7,NaN
5,19463712-1,2023-1,\n \n \n \n ...,4.4,4.9,4.9,5.8,4.0,6.0,4.9,NaN
6,20218430-1,2023-1,\nUNIVERSIDAD CATÓLICA DEL NORTE \nFACULTAD D...,6.1,5.8,5.5,5.0,4.5,5.8,5.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...
177,19928371-1,2021-1,\nUNIVERSIDAD CATÓLICA DEL NORTE \nFACULTAD D...,6.8,6.2,6.0,5.7,6.3,6.0,6.1,NaN
178,19952605-1,2021-1,\n \n \n \nUNIVERSIDAD CATÓLICA DEL NORTE \nF...,7.0,6.8,6.8,7.0,7.0,7.0,6.9,NaN
179,19957163-1,2021-1,\n \nUNIVERSIDAD CATÓLICA DEL NORTE \nFACULTA...,6.5,4.5,5.8,5.5,6.4,6.3,5.8,NaN
180,20180533-1,2021-1,\nUNIVERSIDAD CATÓLICA DEL NORTE \nFACULTAD D...,7.0,6.0,6.7,6.8,6.8,7.0,6.7,NaN


## David Sandbox :)

In [35]:
def get_month_by_number(number: int):
    months = ["enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"]
    return months[number]

months_amount: int = 11


def get_date_start_index_from_document(document: str):
    for i in range(months_amount):
        index: int = document.find(get_month_by_number(i))
        if index != -1:
            return index
    return -1


def get_date_from_document(document: str):
    result: int = get_date_start_index_from_document(document)
    if result != -1:
        return True
    return False

safe_number_provider = 20 # Actually could be not safe

In [48]:
"""# Used for testing purposes, should be safe deleted
haveDate: list = []
for i in range(safe_number_provider):
    result: bool = get_date_from_document(dataset.iloc[i, 2])
    haveDate.append({result, str(i)})
    
    front_pages = []
for i in range(safe_number_provider):
    document: str = dataset.iloc[i,2]
    result: int = get_date_start_index_from_document(document)
    front_pages.append(result)
"""

'# Used for testing purposes, should be safe deleted\nhaveDate: list = []\nfor i in range(safe_number_provider):\n    result: bool = get_date_from_document(dataset.iloc[i, 2])\n    haveDate.append({result, str(i)})\n    \n    front_pages = []\nfor i in range(safe_number_provider):\n    document: str = dataset.iloc[i,2]\n    result: int = get_date_start_index_from_document(document)\n    front_pages.append(result)\n'

In [32]:
"""
id = "19104209-1"
doc = fitz.open(f"dataset/{id}.pdf")
page = doc[1]
words = page.get_text("words", sort=True)
for i, word in enumerate(words):
    text = word[4]
    print(text)
"""

max_block_value = 0
max_page_value = 0
for id in dataset['id']:
    doc = fitz.open(f"dataset/{id}.pdf")
    values = []
    for page in doc:
        blocks: list = page.get_text("blocks")
        values.append([block[4] for block in blocks])
        display(values)
    for value in values:
        value_size = len(value)
        if value_size > max_block_value:
            max_block_value = value_size
    if len(values) > max_page_value:
        max_page_value = len(values)
    break    
max_block_value

48

In [6]:
max_page_value

75

## Count the blocks of the entire dataset

In [37]:
total_values = []
normal_blocks = []
max_values_length = 0
for id in dataset['id']:
    doc = fitz.open(f"dataset/{id}.pdf")
    values = []
    for page in doc:
        blocks: list = page.get_text("blocks")
        flatten_values = []
        phrase = ""
        for block in blocks:
            normal_blocks.append(block[4])
            for element in block[4]:
                phrase = phrase + element
        values += [block[4] for block in blocks]
        total_values.append([block[4] for block in blocks])
    if len(values) > max_values_length:
        max_values_length = len(values)


In [38]:
max_values_length

1782

In [33]:
len(normal_blocks)

123858

In [34]:
len(total_values)

5876

[' \n  \nUNIVERSIDAD CATÓLICA DEL NORTE  \nFACULTAD DE INGENIERÍA Y CIENCIAS GEOLÓGICAS  \nDEPARTAMENTO DE INGENIERÍA DE SISTEMAS Y COMPUTACIÓN  \n \n  \n  \n  \nInforme Práctica Pre Profesional I  \n  \n  \n  \n  \n \n \n \n \n \n \n \nAntofagasta \nMarzo, 2023 \n \n<image: DeviceRGB, width: 161, height: 172, bpc: 8>Nombre Alumno: Diego Alexi Aguilera \nVillanelo  \nRut: 20.908.397-3 \nCarrera: Ingeniería en Computación e \nInformática \nNivel: 7 \nNombre Empresa: UCN-LSA \nCiudad: Antofagasta \nFecha inicio: 3 enero 2023 \nFecha termino: 3 marzo 2023 \n',
 'ii \n \nÍndice General \n        Página     \nÍndice de Figuras ...................................................................................................................... iv \nGlosario ..................................................................................................................................... vi \nResumen ..........................................................................................

In [23]:
max_value = -1
for value in total_values:
    if len(value) > max_value:
        max_value = len(value)
max_value

165

In [18]:
doc = fitz.open("dataset/18178688-1.pdf")
blocks: list = doc[9].get_text("blocks")

aaaa = [block[4] for block in blocks]
aaaa

[' \n \n \n',
 '3 \n \n',
 'II. \nTRABAJO REALIZADO  \n',
 ' \n',
 'Al comenzar la práctica se dio acceso a una carpeta situada en Google Drive la cual contenía \n',
 'la versión de Moodle 3.8.2, en la cual fue realizado el trabajo solicitado por la empresa. \n',
 'Se comenzó el proceso de investigación e implementación simultáneamente mediante el \n',
 'estudio de la documentación de la plataforma Moodle y con el apoyo de video tutoriales de \n',
 'cómo utilizar la herramienta con el objetivo de agilizar el procedimiento. \n',
 ' \n',
 '2.1.      Descripción de la Problemática (Moodle) \n',
 ' \n',
 'La principal problemática es que, debido a la situación actual en la que se encuentra el país, se \n',
 'redujo el índice de ventas de cursos de capacitación y arriendos de salas de clases. Por esto se \n',
 'realizó una investigación respecto a la metodología E-Learning y que herramienta gratuita \n',
 'ayudaría mejor a la implementación de esta metodología llegando a la conclusión de qu

In [39]:
non_repeated_values = []

for value in values:
    if value not in non_repeated_values:
        non_repeated_values.append(value)

# Removed because Keras use this as reserved Token
non_repeated_values.remove('')

len(non_repeated_values)

5851

In [40]:
import tensorflow as tf

max_features = len(non_repeated_values) + 2
max_len = None 


vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len,
 vocabulary=non_repeated_values)

vectorize_layer.get_vocabulary(include_special_tokens=True)

['',
 '[UNK]',
 ' \n  \nUNIVERSIDAD CATÓLICA DEL NORTE  \nFACULTAD DE INGENIERÍA Y CIENCIAS GEOLÓGICAS  \nDEPARTAMENTO DE INGENIERÍA DE SISTEMAS Y COMPUTACIÓN  \n \n  \n  \n  \nInforme Práctica Pre Profesional I  \n  \n  \n  \n  \n \n \n \n \n \n \n \nAntofagasta \nMarzo, 2023 \n \n<image: DeviceRGB, width: 161, height: 172, bpc: 8>Nombre Alumno: Diego Alexi Aguilera \nVillanelo  \nRut: 20.908.397-3 \nCarrera: Ingeniería en Computación e \nInformática \nNivel: 7 \nNombre Empresa: UCN-LSA \nCiudad: Antofagasta \nFecha inicio: 3 enero 2023 \nFecha termino: 3 marzo 2023 \n',
 'ii \n \nÍndice General \n        Página     \nÍndice de Figuras ...................................................................................................................... iv \nGlosario ..................................................................................................................................... vi \nResumen ...........................................................................